In [ ]:
import pandas as pd
from google.colab import files
import re

print("📊 Excel to Odoo CSV Converter - Simple & Direct")
print("=" * 50)

# === 1. Upload Excel file ===
print("Upload your Excel file containing invoice data:")
uploaded = files.upload()
excel_path = list(uploaded.keys())[0]
print(f"Processing: {excel_path}")

# === 2. Read Excel file ===
df = pd.read_excel(excel_path)
print(f"Loaded {len(df)} rows from Excel")
print(f"Columns found: {list(df.columns)}")

# Display first few rows to understand structure
print("\n📋 First 3 rows of your data:")
print(df.head(3))

# === 3. Clean & normalize data ===
print("\n🧹 Cleaning data...")

# Handle common column name variations
column_mapping = {}
for col in df.columns:
    col_lower = col.lower().strip()
    if any(word in col_lower for word in ['date', 'tanggal']):
        column_mapping['DATE'] = col
    elif any(word in col_lower for word in ['bill', 'invoice', 'no', 'nomor']):
        column_mapping['BILL_NO'] = col
    elif any(word in col_lower for word in ['supplier', 'vendor', 'pemasok']):
        column_mapping['SUPPLIER'] = col
    elif any(word in col_lower for word in ['npwp', 'tax', 'pajak']):
        column_mapping['NPWP'] = col
    elif any(word in col_lower for word in ['description', 'desc', 'keterangan', 'item']):
        column_mapping['DESCRIPTION'] = col
    elif any(word in col_lower for word in ['qty', 'quantity', 'jumlah']):
        column_mapping['QTY'] = col
    elif any(word in col_lower for word in ['rate', 'price', 'unit', 'harga']):
        column_mapping['RATE'] = col
    elif any(word in col_lower for word in ['amount', 'total', 'nilai']):
        column_mapping['AMOUNT'] = col

print(f"Column mapping: {column_mapping}")

# Rename columns to standard names
for std_name, orig_name in column_mapping.items():
    if orig_name in df.columns:
        df[std_name] = df[orig_name]

# Clean numeric columns
if 'QTY' in df.columns:
    df["QTY"] = pd.to_numeric(df["QTY"], errors="coerce").fillna(1)
else:
    df["QTY"] = 1

if 'RATE' in df.columns:
    # Remove commas, dots used as thousand separators, currency symbols
    df["RATE"] = df["RATE"].astype(str).str.replace(r'[Rp,.]', '', regex=True).str.strip()
    df["RATE"] = pd.to_numeric(df["RATE"], errors="coerce").fillna(0)
else:
    df["RATE"] = 0

if 'AMOUNT' in df.columns:
    df["AMOUNT"] = df["AMOUNT"].astype(str).str.replace(r'[Rp,.]', '', regex=True).str.strip()
    df["AMOUNT"] = pd.to_numeric(df["AMOUNT"], errors="coerce").fillna(0)
else:
    df["AMOUNT"] = df["QTY"] * df["RATE"]

# Clean text fields
if 'SUPPLIER' in df.columns:
    df["SUPPLIER"] = df["SUPPLIER"].astype(str).str.strip()
    # Remove common extra text
    df["SUPPLIER"] = df["SUPPLIER"].str.replace("ORIGINAL FOR RECIPIENT", "").str.strip()
else:
    df["SUPPLIER"] = "Unknown Supplier"

if 'NPWP' in df.columns:
    df["NPWP"] = df["NPWP"].astype(str).str.replace(r'[^\d]', '', regex=True)
    df["NPWP"] = df["NPWP"].str.zfill(15)  # Standard NPWP length
else:
    df["NPWP"] = ""

if 'DESCRIPTION' not in df.columns:
    df["DESCRIPTION"] = "Invoice Item"

if 'DATE' not in df.columns:
    from datetime import datetime
    df["DATE"] = datetime.now().strftime('%Y-%m-%d')

if 'BILL_NO' not in df.columns:
    df["BILL_NO"] = df.index.map(lambda x: f"INV-{x+1:04d}")

print(f"✅ Data cleaned. {len(df)} rows ready for processing")

# === 4. Create Odoo Import Format ===
print("\n🔄 Converting to Odoo format...")

# Calculate price excluding tax (assume 11% PPN if AMOUNT > RATE)
df["PRICE_EXCL_TAX"] = df.apply(lambda row:
    row["AMOUNT"] / 1.11 if row["AMOUNT"] > row["RATE"] * row["QTY"] * 1.05
    else row["RATE"], axis=1
).round(2)

# Create Odoo CSV structure
odoo_df = pd.DataFrame({
    "partner_id/name": df["SUPPLIER"],
    "partner_id/vat": df["NPWP"],
    "ref": df["BILL_NO"],
    "invoice_date": df["DATE"],
    "invoice_date_due": df["DATE"],  # Same as invoice date, adjust if needed
    "move_type": "in_invoice",
    "currency_id/name": "IDR",
    "invoice_line_ids/name": df["DESCRIPTION"].str[:64],  # Limit to 64 chars
    "invoice_line_ids/quantity": df["QTY"],
    "invoice_line_ids/price_unit": df["PRICE_EXCL_TAX"],
    "invoice_line_ids/account_id/code": "400000",  # Generic expense account
    "invoice_line_ids/tax_ids/name": df["AMOUNT"].apply(
        lambda x: "11% Purchase" if x > 0 else ""
    )
})

# === 5. Filter valid rows ===
# Remove rows with zero amounts or invalid data
odoo_df = odoo_df[
    (odoo_df["invoice_line_ids/price_unit"] > 0) &
    (odoo_df["partner_id/name"].notna()) &
    (odoo_df["partner_id/name"] != "nan") &
    (odoo_df["partner_id/name"] != "Unknown Supplier")
]

print(f"✅ Created {len(odoo_df)} valid invoice lines")

# === 6. Create multiple format options ===
formats_created = []

# Format 1: Standard format
output_file_1 = excel_path.replace(".xlsx", "_odoo_standard.csv").replace(".xls", "_odoo_standard.csv")
odoo_df.to_csv(output_file_1, index=False, encoding="utf-8-sig")
formats_created.append(output_file_1)

# Format 2: Minimal fields (for problematic imports)
minimal_df = odoo_df[["partner_id/name", "ref", "invoice_date", "move_type",
                     "invoice_line_ids/name", "invoice_line_ids/price_unit"]].copy()
output_file_2 = excel_path.replace(".xlsx", "_odoo_minimal.csv").replace(".xls", "_odoo_minimal.csv")
minimal_df.to_csv(output_file_2, index=False, encoding="utf-8-sig")
formats_created.append(output_file_2)

# Format 3: External ID references
external_df = odoo_df.copy()
external_df["currency_id"] = "base.IDR"
external_df["invoice_line_ids/account_id"] = "account.a_expense"
external_df["invoice_line_ids/tax_ids"] = external_df["invoice_line_ids/tax_ids/name"].apply(
    lambda x: "account.tax_purchase_11" if x else ""
)
# Remove the /name and /code suffixes
external_df = external_df.drop(["currency_id/name", "invoice_line_ids/account_id/code",
                               "invoice_line_ids/tax_ids/name"], axis=1)
output_file_3 = excel_path.replace(".xlsx", "_odoo_external_id.csv").replace(".xls", "_odoo_external_id.csv")
external_df.to_csv(output_file_3, index=False, encoding="utf-8-sig")
formats_created.append(output_file_3)

# === 7. Display results and download ===
print(f"\n📊 CONVERSION SUMMARY:")
print(f"📁 Original file: {excel_path}")
print(f"📋 Rows processed: {len(df)}")
print(f"✅ Valid invoice lines: {len(odoo_df)}")
print(f"💰 Total amount: IDR {odoo_df['invoice_line_ids/price_unit'].sum():,.2f}")

print(f"\n📄 Sample of converted data:")
print(odoo_df.head(2))

print(f"\n📦 Created {len(formats_created)} CSV formats:")
for i, file_path in enumerate(formats_created, 1):
    print(f"{i}. {file_path}")

# Download all files
print(f"\n💾 Downloading files...")
for file_path in formats_created:
    files.download(file_path)
    print(f"✅ Downloaded: {file_path}")

print(f"\n🎯 IMPORT INSTRUCTIONS:")
print(f"1. Try 'standard' format first")
print(f"2. If constraint errors occur, try 'minimal' format")
print(f"3. If field reference errors, try 'external_id' format")
print(f"4. Import via: Accounting → Vendors → Bills → Import")
print(f"\n✅ Conversion complete! Files ready for Odoo import.")